# Store all saved Spotify Songs into a Playlist

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
from getpass import getpass

In [5]:
import datetime

Create a Spotify app at [Spotify's Developer Page](https://developer.spotify.com/dashboard/) and setup your app. Especially, whitelist the redirect-URL below.

Then, enter your own client ID and client secret.

If the redirect URL is port 8080 on localhost, you don't even have to copy-paste the redirect URL.

In [6]:
# Mirko's Test App
APP_CLIENT_ID = "1cae0549a06b47e9ab69ae5d6b9b8d9e"
APP_CLIENT_SECRET = getpass()
REDIRECT_URL = "http://localhost:8080"
REQUEST_LIMIT=50

········


## Documentation

[Spotipy Github Repo](https://github.com/plamere/spotipy)

[Spotipy Documentation](https://spotipy.readthedocs.io/en/latest/)

[Authorization Scopes](https://developer.spotify.com/documentation/general/guides/scopes/)

In [7]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=APP_CLIENT_ID,
                                               client_secret=APP_CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URL,
                                               scope="user-library-read playlist-modify-private playlist-modify-public"))

In [8]:
user = sp.current_user()
user

{'display_name': 'mirkokrause.krause',
 'external_urls': {'spotify': 'https://open.spotify.com/user/mirkokrause.krause'},
 'followers': {'href': None, 'total': 21},
 'href': 'https://api.spotify.com/v1/users/mirkokrause.krause',
 'id': 'mirkokrause.krause',
 'images': [],
 'type': 'user',
 'uri': 'spotify:user:mirkokrause.krause'}

In [9]:
def get_saved_tracks(limit=REQUEST_LIMIT):
    saved_tracks = []
    offset = 0
    while True:
        tracks = pd.DataFrame(sp.current_user_saved_tracks(limit=limit, offset=offset))
        if len(tracks) > 0:
            saved_tracks.append(tracks)
            print("got song %d to %d" % (offset + 1, offset + limit))
            offset += limit
        else:
            saved_tracks = pd.concat(saved_tracks).reset_index(drop=True)
            break
    saved_tracks = saved_tracks['items'].apply(pd.Series)['track'].apply(pd.Series)
    saved_tracks = saved_tracks.set_index('id')
    print("got %d saved songs" % len(saved_tracks))
    return saved_tracks

In [10]:
saved_tracks = get_saved_tracks()
saved_tracks.head(0)

got song 1 to 50
got song 51 to 100
got song 101 to 150
got song 151 to 200
got song 201 to 250
got song 251 to 300
got song 301 to 350
got song 351 to 400
got song 401 to 450
got song 451 to 500
got song 501 to 550
got song 551 to 600
got song 601 to 650
got song 651 to 700
got song 701 to 750
got song 751 to 800
got song 801 to 850
got song 851 to 900
got song 901 to 950
got song 951 to 1000
got song 1001 to 1050
got song 1051 to 1100
got song 1101 to 1150
got song 1151 to 1200
got song 1201 to 1250
got song 1251 to 1300
got song 1301 to 1350
got song 1351 to 1400
got song 1401 to 1450
got song 1451 to 1500
got song 1501 to 1550
got song 1551 to 1600
got song 1601 to 1650
got song 1651 to 1700
got song 1701 to 1750
got song 1751 to 1800
got song 1801 to 1850
got song 1851 to 1900
got song 1901 to 1950
got song 1951 to 2000
got song 2001 to 2050
got song 2051 to 2100
got song 2101 to 2150
got 2122 saved songs


,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,is_local,name,popularity,preview_url,track_number,type,uri
id,,,,,,,,,,,,,,,,


In [11]:
def get_saved_track_artists(saved_tracks):
    saved_track_artists = saved_tracks.explode('artists')['artists'].apply(pd.Series)
    return saved_track_artists

In [12]:
saved_track_artists = get_saved_track_artists(saved_tracks)
saved_track_artists.head(0)

,external_urls,href,id,name,type,uri
id,,,,,,


In [13]:
def get_playlists(limit=REQUEST_LIMIT):
    playlists = []
    offset=0
    while True:
        p = pd.DataFrame(sp.current_user_playlists(limit=limit, offset=offset))
        if len(p) > 0:
            playlists.append(p)
            print("got playlists %d to %d" % (offset + 1, offset + limit))
            offset += limit
        else:
            playlists = pd.concat(playlists).reset_index(drop=True)
            break
    playlists = playlists['items'].apply(pd.Series).set_index('id')
    print("got %d playlists" % len(playlists))
    return playlists

In [14]:
playlists = get_playlists()
playlists.head(0)

got playlists 1 to 50
got playlists 51 to 100
got 52 playlists


,collaborative,description,external_urls,href,images,name,owner,primary_color,public,snapshot_id,tracks,type,uri
id,,,,,,,,,,,,,


In [15]:
def add_tracks_to_playlist(playlist_id: str, tracks: list, limit: int = REQUEST_LIMIT):
    offset = 0
    while True:
        t = tracks[offset:offset + limit]
        if len(t) > 0:
            sp.playlist_add_items(playlist_id=playlist_id, items=t)
            print("added song items %d to %d to playlist %s" % (offset + 1, offset + limit, playlist_id))
            offset += limit
        else:
            break

In [16]:
def backup_saved_songs(user_id=user['id'], playlist_name=None):
    if playlist_name is None:
        playlist_name = datetime.datetime.now().strftime('%g%m%d')
    saved_tracks = get_saved_tracks()
    new_playlist = sp.user_playlist_create(user=user['id'], name=playlist_name)
    track_ids = list(saved_tracks.index)
    playlist_id = new_playlist['id']
    add_tracks_to_playlist(playlist_id=playlist_id, tracks=track_ids)
    print("added %d songs to playlist \"%s\"" % (len(track_ids), playlist_name))

In [17]:
backup_saved_songs()

got song 1 to 50
got song 51 to 100
got song 101 to 150
got song 151 to 200
got song 201 to 250
got song 251 to 300
got song 301 to 350
got song 351 to 400
got song 401 to 450
got song 451 to 500
got song 501 to 550
got song 551 to 600
got song 601 to 650
got song 651 to 700
got song 701 to 750
got song 751 to 800
got song 801 to 850
got song 851 to 900
got song 901 to 950
got song 951 to 1000
got song 1001 to 1050
got song 1051 to 1100
got song 1101 to 1150
got song 1151 to 1200
got song 1201 to 1250
got song 1251 to 1300
got song 1301 to 1350
got song 1351 to 1400
got song 1401 to 1450
got song 1451 to 1500
got song 1501 to 1550
got song 1551 to 1600
got song 1601 to 1650
got song 1651 to 1700
got song 1701 to 1750
got song 1751 to 1800
got song 1801 to 1850
got song 1851 to 1900
got song 1901 to 1950
got song 1951 to 2000
got song 2001 to 2050
got song 2051 to 2100
got song 2101 to 2150
got 2122 saved songs
added song items 1 to 50 to playlist 14VsdYlXq5X6R87GGBGSDp
added song items